In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend
import os
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
print(os.listdir("/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/"))

In [4]:
img_yes = load_img('/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/yes/Y11.jpg')
fig = plt.figure(figsize=(16,8))
tumor = fig.add_subplot(1,3,1)
plt.imshow(img_yes)
tumor.set_xlabel('tumor')

img_no = load_img('/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/no/N3.jpg')
normal = fig.add_subplot(1,3,2)
plt.imshow(img_no)
normal.set_xlabel("Normal")
plt.show()

In [5]:
#set dimenstion for images
img_height = 256
img_width = 256

#set all samples and epochs limit
train_samples = 253
val_samples = 10
epochs = 30
batch_size = 1
input_shape = (img_width, img_height, 3)

In [6]:
# Train_data_augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [7]:
# only rescaling data for test_set
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
train_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/"

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [10]:
# only 25 image in validation
validation_generator = test_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=8,
    class_mode='binary')

In [11]:
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
model.fit(train_generator,
                    epochs = 40,
                    validation_data=validation_generator,
                    verbose =1 
                   )

In [15]:
predictions = model.predict(validation_generator)
predictions = np.round(predictions).tolist()

y_true = validation_generator.labels
y_pred = predictions

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred))

In [17]:
print(classification_report(y_true, y_pred))

In [18]:
model.evaluate(validation_generator)

In [19]:
model.save('tumor.h5')